# Model training notebook


In [1]:
from LangDataloader import LangDataloader
from LangDataloader import addData
import numpy as np
import tensorflow as tf
import keras_lmu 
import copy
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
defultePlotSize = plt.rcParams['figure.figsize']
oringalFontSize = matplotlib.rcParams['font.size']

2023-05-26 14:23:03.553289: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


We will only load a subset of the total dataset to save time. 

In [2]:
data = LangDataloader("/home/khood/GitHub/STREAM/Prepocessing/processedData/index.csv", "/home/khood/GitHub/STREAM/Prepocessing/processedData")
rawData = []
lastStep = 0
labelToNum = {}
num = 0
for step in list(range(0,len(data),100))[:11]:
    for file in range(len(data))[lastStep:step]:
        print(f"adding {lastStep} to {step} of {len(data)}: {'{0:.0f}%'.format(step/len(data) * 100)}... ",end="\x1b\r")
        label = None
        if data[file][1][0] in labelToNum.keys():
            label = np.array([labelToNum[data[file][1][0]]])
        else:
            labelToNum[data[file][1][0]] = num
            num = num + 1
            label = np.array([labelToNum[data[file][1][0]]])
            
        melspectrogram = np.array([data[file][0]]).flatten()
        melspectrogram = [[i] for i in melspectrogram]
        rawData.append([melspectrogram, label])
        
    lastStep = step
        
# for file in range(len(data))[lastStep:]: # grab the remaning (if the size%stepSize != 0 then there will be some left over)
#     print(f"adding {lastStep} to {len(data)} of {len(data)}: {'{0:.0f}%'.format(step/len(data) * 100)}... ",end="\x1b\r")
#     rawData.append(data[file])
    
rawData[0][0]

[[3.755148272515285e-20],
 [2.568578216875089e-13],
 [8.811583029455505e-06],
 [0.00012983147462364286],
 [0.0001186872468679212],
 [0.00010715652024373412],
 [0.00017705878417473286],
 [0.00019923163927160203],
 [9.357128874398768e-05],
 [1.4269797247834504e-05],
 [1.2840345334552694e-05],
 [1.3032259630563203e-05],
 [2.3437069103238173e-05],
 [2.5691699192975648e-05],
 [5.593668538494967e-05],
 [1.4664220543636475e-05],
 [1.117618739954196e-05],
 [7.151053068810143e-06],
 [1.049544516718015e-05],
 [1.284125119127566e-05],
 [1.91502804227639e-05],
 [1.3268237125885207e-05],
 [5.238910034677247e-06],
 [3.8975489587755874e-05],
 [0.0014837041962891817],
 [0.0024472600780427456],
 [0.0019210706232115626],
 [0.005382259376347065],
 [0.007959770038723946],
 [0.006091925781220198],
 [0.00456344336271286],
 [0.02438884787261486],
 [0.03551292419433594],
 [0.06724800169467926],
 [0.027123361825942993],
 [0.006219853647053242],
 [0.05725622549653053],
 [0.07347908616065979],
 [0.00977116916328

convert targets to numbers classes

In [3]:
len(labelToNum.keys())

176

The last dimension should be time, the second to last should be the n_mels

<p>not flattened shape: (1, 128, 430)
<p>flattened shape: (55040,)

In [4]:
# rawData

In [5]:
# print(rawData[0].shape)
# np.array(rawData[0][0]).flatten().shape

In [6]:
n_pixels = 430

lmu_layer = keras_lmu.LMU(
    memory_d=1,
    order=256,
    theta=n_pixels,
    hidden_cell=tf.keras.layers.SimpleRNNCell(212),
    hidden_to_memory=False,
    memory_to_memory=False,
    input_to_hidden=True,
    kernel_initializer="ones",
)

# TensorFlow layer definition
inputs = tf.keras.Input((n_pixels, 1))
lmus = lmu_layer(inputs)
outputs = tf.keras.layers.Dense(len(labelToNum.keys()))(lmus)

# TensorFlow model definition
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"],
)
model.summary()

2023-05-26 14:24:09.463415: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 14:24:09.468076: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 430, 1)]          0         
                                                                 
 lmu (LMU)                   (None, 212)               99641     
                                                                 
 dense (Dense)               (None, 176)               37488     
                                                                 
Total params: 137,129
Trainable params: 137,129
Non-trainable params: 0
_________________________________________________________________


In [7]:
# rawData[0][0].shape

In [8]:
valid = []
train = []
addData(valid, train, rawData, int(len(rawData)*0.9))
print(f"Train: {len(train)}")
print(f"Valid: {len(valid)}")

Train: 900
Valid: 100


In [9]:
X_train = []
Y_train = []
for i in train:
    X_train.append(i[0])
    Y_train.append(i[0][0])

In [10]:
print(X_train[0])
len(X_train[0])

[[3.755148272515285e-20], [2.568578216875089e-13], [8.811583029455505e-06], [0.00012983147462364286], [0.0001186872468679212], [0.00010715652024373412], [0.00017705878417473286], [0.00019923163927160203], [9.357128874398768e-05], [1.4269797247834504e-05], [1.2840345334552694e-05], [1.3032259630563203e-05], [2.3437069103238173e-05], [2.5691699192975648e-05], [5.593668538494967e-05], [1.4664220543636475e-05], [1.117618739954196e-05], [7.151053068810143e-06], [1.049544516718015e-05], [1.284125119127566e-05], [1.91502804227639e-05], [1.3268237125885207e-05], [5.238910034677247e-06], [3.8975489587755874e-05], [0.0014837041962891817], [0.0024472600780427456], [0.0019210706232115626], [0.005382259376347065], [0.007959770038723946], [0.006091925781220198], [0.00456344336271286], [0.02438884787261486], [0.03551292419433594], [0.06724800169467926], [0.027123361825942993], [0.006219853647053242], [0.05725622549653053], [0.07347908616065979], [0.009771169163286686], [0.004557274281978607], [0.0039

55040

In [11]:
X_valid = []
Y_valid = []
for i in valid:
    X_valid.append(i[0])
    Y_valid.append(i[0][0])

In [12]:
do_training = True
batch_size = 100
epochs = 10

saved_weights_fname = "./psMNIST-weights.hdf5"
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=saved_weights_fname, monitor="val_loss", verbose=1, save_best_only=True
    ),
]

result = None
if do_training:
    result = model.fit(
        X_train,
        Y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_valid, Y_valid),
        callbacks=callbacks,
    )

Epoch 1/10


ValueError: in user code:

    File "/home/khood/anaconda3/envs/keras_lmu/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/khood/anaconda3/envs/keras_lmu/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/khood/anaconda3/envs/keras_lmu/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/khood/anaconda3/envs/keras_lmu/lib/python3.9/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/home/khood/anaconda3/envs/keras_lmu/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/khood/anaconda3/envs/keras_lmu/lib/python3.9/si